In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as et
import requests

# Comment out below as appopriate if from a url or from a local file

In [2]:
# from url
url = 'https://raw.githubusercontent.com/iKhaled/MCFCAnalytics/master/Bolton_ManCityF24.xml'
root = et.fromstring(requests.get(url).content)
# from file
#tree = et.parse('country_data.xml')
#root = tree.getroot()

# Parse the data into a list of dictionaries

In [3]:
events = []
qualifiers = []
for game in root:
    game_id = root[0].attrib.get('id')
    for event in game:
        type_id = event.attrib.get('type_id')
        event_id = event.attrib.get('id')
        team_event_id = event.attrib.get('event_id')
        player_id = event.attrib.get('player_id')
        new_event = {'game_id':game_id,
                     'event_id':event_id,
                     'team_event_id':team_event_id,
                     'player_id':player_id,
                     'type_id':type_id,
                     'team_id':event.attrib.get('team_id'),
                     'period_id':event.attrib.get('period_id'),
                     'min':event.attrib.get('min'),
                     'sec':event.attrib.get('sec'),
                     'outcome':event.attrib.get('outcome'),
                     'x':event.attrib.get('x'),
                     'y':event.attrib.get('y'),
                     'timestamp':event.attrib.get('timestamp')}
        events.append(new_event)
        for qualifier in event:
            qualifier_id = qualifier.attrib.get('qualifier_id')
            value = qualifier.attrib.get('value')
            if value is None:
                value = True
            new_qualifier = {'event_id':event_id, 'qualifier_id': qualifier_id, 'value': value}
            qualifiers.append(new_qualifier)

# Create events dataframe

In [4]:
df_events = pd.DataFrame(events)
# reorder columns
df_events = df_events[['game_id','team_event_id','event_id','team_id','player_id',
                       'period_id','min','sec','timestamp','type_id','outcome','x','y']].copy()

# Map the event types to event names

In [5]:
# map event types to event names
opta_event_map = {'1': 'pass', 
                  '5': 'out_of_play', 
                  '49': 'ball_recovery',
                  '4': 'foul', 
                  '44': 'aerial_duel', 
                  '61': 'ball_touch_lose_ball', 
                  '3': 'take_on', 
                  '7': 'tackle', 
                  '12': 'clearance', 
                  '43': 'deleted_event', 
                  '8': 'interception', 
                  '50': 'dispossessed', 
                  '45': 'challenge', 
                  '6': 'corner_awarded', 
                  '74': 'blocked_pass', 
                  '52': 'goalkeeper_pick-up', 
                  '15': 'shot_attempt_saved', 
                  '10': 'goalkeeper_save', 
                  '13': 'shot_miss', 
                  '30': 'end_of_period', 
                  '18': 'player_off', 
                  '19': 'player_on', 
                  '17': 'card', 
                  '55': 'offside_provoked', 
                  '2': 'pass_offside', 
                  '32': 'start_of_period', 
                  '28': 'end_delay', 
                  '27': 'start_delay', 
                  '16': 'shot_goal', 
                  '40': 'formation_change', 
                  '37': 'collection_end', 
                  '34': 'team_set_up', 
                  '70': 'injury_time_announcement', 
                  '41': 'goalkeeper_punch', 
                  '24': 'condition_change', 
                  '11': 'goalkeeper_claim', 
                  '59': 'goalkeeper_sweeper', 
                  '51': 'error_lose_ball', 
                  '68': 'referee_drop_ball', 
                  '56': 'shield_ball_opp', 
                  '58': 'goalkeeper_penalty_faced', 
                  '65': 'contentious_referee_decision', 
                  '14': 'shot_ball_hits_post', 
                  '54': 'goalkeeper_smother', 
                  '42': 'good_skill', 
                  '53': 'goalkeeper_cross_not_claimed', 
                  '71': 'coach_setup', 
                  '60': 'chance_missed'}
df_events['type'] = df_events.type_id.astype(str).map(opta_event_map)

# Create qualifiers dataframe

In [6]:
qualifiers = pd.DataFrame(qualifiers)
qualifiers = qualifiers.pivot(index='event_id',columns='qualifier_id',values='value')    

# Map the qualifiers to events

In [7]:
opta_qualifiers = {'56': 'zone', 
                   '140': 'pass_end_x', 
                   '141': 'pass_end_y', 
                   '212': 'length', 
                   '213': 'angle', 
                   '233': 'opposite_related_event_id', 
                   '1': 'long_ball', 
                   '286': 'duel_events_offensive', 
                   '285': 'duel_events_defensive', 
                   '155': 'chipped', 
                   '152': 'direct', 
                   '13': 'foul', 
                   '3': 'head_pass', 
                   '107': 'throw_in', 
                   '157': 'launch', 
                   '2': 'cross', 
                   '5': 'free_kick_taken', 
                   '265': 'attempted_tackle', 
                   '55': 'related_event_id', 
                   '15': 'head', 
                   '156': 'lay_off', 
                   '236': 'blocked_pass', 
                   '22': 'regular_play', 
                   '154': 'intentional_assist', 
                   '102': 'goal_mouth_y_coordinate', 
                   '103': 'goal_mouth_z_coordinate', 
                   '124': 'goalkeeper_goal_kick', 
                   '73': 'shot_left_missed', 
                   '29': 'assisted', 
                   '167': 'out_of_play', 
                   '44': 'player_position', 
                   '59': 'jersey_number', 
                   '210': 'assist', 
                   '74': 'shot_high_missed', 
                   '20': 'right_footed', 
                   '146': 'blocked_x_coordinate', 
                   '147': 'blocked_y_coordinate', 
                   '295': 'shirt_pull_holding', 
                   '230': 'goalkeeper_x_coordinate', 
                   '231': 'goalkeeper_y_coordinate', 
                   '215': 'individual_play', 
                   '168': 'flick_on', 
                   '328': 'first_touch', 
                   '42': 'tactical', 
                   '196': 'switch_of_play', 
                   '294': 'shove_push', 
                   '6': 'corner_taken', 
                   '18': 'shot_location_out_of_boxcentre', 
                   '17': 'shot_location_boxcentre', 
                   '123': 'goalkeeper_throw', 
                   '72': 'left_footed', 
                   '75': 'shot_right_missed', 
                   '82': 'shot_blocked', 
                   '57': 'end_type', 
                   '94': 'def_block', 
                   '145': 'formation_slot', 
                   '292': 'detailed_position_id', 
                   '293': 'position_side_id', 
                   '241': 'indirect', 
                   '211': 'overrun', 
                   '31': 'yellow_card', 
                   '279': 'kick_off', 
                   '7': 'players_caught_offside', 
                   '237': 'goalkeeper_low_goal_kick', 
                   '30': 'involved', 
                   '130': 'team_formation', 
                   '131': 'team_player_formation', 
                   '194': 'captain', 
                   '182': 'goalkeeper_saves_with_hands', 
                   '127': 'direction_of_play', 
                   '209': 'game_end', 
                   '223': 'in_swinger', 
                   '25': 'from_corner', 
                   '185': 'blocked_cross', 
                   '224': 'out_swinger', 
                   '179': 'goalkeeper_save_while_diving', 
                   '199': 'gk_kick_from_hands', 
                   '214': 'big_chance', 
                   '78': 'shot_low_centre', 
                   '41': 'injury', 
                   '264': 'aerial_foul', 
                   '10': 'foul_hand', 
                   '63': 'shot_location_box_right', 
                   '173': 'goalkeeper_parried_safe', 
                   '287': 'goalkeeper_over_arm', 
                   '24': 'set_piece', 
                   '197': 'team_kit', 
                   '227': 'resume', 
                   '9': 'penalty', 
                   '64': 'shot_location_box_left', 
                   '4': 'through_ball', 
                   '108': 'volley', 
                   '76': 'shot_low_left', 
                   '238': 'fair_play', 
                   '277': 'minutes', 
                   '80': 'shot_low_right', 
                   '198': 'goalkeeper_hoof', 
                   '153': 'not_past_goal_line', 
                   '53': 'injured_player_id', 
                   '81': 'shot_high_right', 
                   '177': 'goalkeeper_collected', 
                   '174': 'goalkeeper_parried_danger', 
                   '220': 'player_on_near_post',
                   '221': 'player_on_far_post',
                   '222': 'no_players_on_posts',
                   '117': 'lob',
                   '118': 'one_bounce',
                   '229': 'Post-match complete',
                   '77': 'shot_high_left', 
                   '26': 'free_kick', 
                   '46': 'conditions', 
                   '47': 'field_pitch', 
                   '49': 'attendance_figure', 
                   '121': 'swerve_right', 
                   '255': 'open_roof', 
                   '256': 'air_humidity', 
                   '257': 'air_pressure', 
                   '259': 'celsius_degrees', 
                   '84': 'shot_close_right', 
                   '120': 'swerve_left', 
                   '180': 'goalkeeper_save_while_stooping', 
                   '184': 'dissent', 
                   '88': 'goalkeeper_high_claim', 
                   '83': 'shot_close_left', 
                   '178': 'goalkeeper_save_while_standing', 
                   '278': 'tap', 
                   '79': 'shot_high_centre', 
                   '195': 'pull_back', 
                   '16': 'shot_location_small_boxcentre', 
                   '85': 'shot_close_high', 
                   '100': 'shot_six_yard_blocked', 
                   '183': 'goalkeeper_saves_with_feet', 
                   '136': 'goalkeeper_touched', 
                   '12': 'foul_dangerous_play', 
                   '89': 'goalkeeper_1_on_1', 
                   '114': 'weak', 
                   '61': 'shot_location_small_boxleft', 
                   '280': 'fantasy_assist_type', 
                   '281': 'fantasy_assisted_by', 
                   '282': 'fantasy_assist_team', 
                   '35': 'argument', 
                   '319': 'captain_change', 
                   '133': 'deflection', 
                   '170': 'leading_to_goal', 
                   '240': 'gk_start', 
                   '176': 'goalkeeper_caught', 
                   '8': 'goal_disallowed', 
                   '14': 'last_line', 
                   '60': 'shot_location_small_boxright', 
                   '113': 'strong', 
                   '132': 'dive', 
                   '186': 'penalty_scored', 
                   '232': 'goalkeeper_smother_unchallenged', 
                   '23': 'fast_break', 
                   '37': 'time_wasting', 
                   '62': 'shot_location_box_deepright', 
                   '65': 'shot_location_box_deepleft', 
                   '138': 'hit_woodwork', 
                   '169': 'leading_to_attempt', 
                   '239': 'by_wall', 
                   '254': 'follows_a_dribble', 
                   '139': 'goalkeeper_save_shot_from_own_player', 
                   '216': '2nd_related_event_id', 
                   '217': '2nd_assisted', 
                   '218': '2nd_assist', 
                   '33': 'red_card', 
                   '19': 'shot_location_35_plus_centre', 
                   '32': 'second_yellow', 
                   '181': 'goalkeeper_save_while_reaching', 
                   '66': 'shot_location_out_of_box_deepright', 
                   '87': 'shot_close_right_and_high', 
                   '21': 'other_body_part', 
                   '225': 'straight', 
                   '247': 'offside', 
                   '290': 'coach_types', 
                   '314': 'end_of_offside', 
                   '69': 'shot_location_out_of_boxdeep_left', 
                   '160': 'throw_in_set_piece', 
                   '201': 'referee_delay', 
                   '275': 'goalkeeper_save_hit_bar', 
                   '28': 'own_goal', 
                   '86': 'shot_close_left_and_high', 
                   '101': 'shot_saved_off_line', 
                   '175': 'goalkeeper_fingertip', 
                   '187': 'penalty_saved', 
                   '191': 'off_the_ball_foul', 
                   '228': 'own_shot_blocked', 
                   '246': 'drinks_break', 
                   '273': 'goalkeeper_save_hit_right_post', 
                   '297': 'offside_follows_shot_rebound', 
                   '70': 'shot_location_35_plus_right', 
                   '122': 'swerve_moving', 
                   '137': 'goalkeeper_saved_off_target_shot', 
                   '159': 'foul_and_abusive_language', 
                   '190': 'goalkeeper_saved_from_shot_off_target', 
                   '274': 'goalkeeper_save_hit_left_post', 
                   '298': 'offside_follows_shot_blocked', 
                   '68': 'shot_location_out_of_boxleft', 
                   '158': 'persistent_infringement', 
                   '161': 'encroachment', 
                   '163': 'entering_field', 
                   '205': 'object_thrown_on_pitch', 
                   '283': 'coach_id', 
                   '300': 'solo_run', 
                   '36': 'violent_conduct', 
                   '38': 'excessive_celebration', 
                   '40': 'other_reason', 
                   '67': 'shot_location_out_of_boxright', 
                   '71': 'shot_location_35_plus_left', 
                   '165': 'professional_foul_last_man', 
                   '188': 'penalty_missed', 
                   '189': 'not_visible', 
                   '200': 'referee_stop', 
                   '263': 'direct_corner', 
                   '276': 'out_on_sideline'}
# rename the qualifiers
qualifiers.rename(opta_qualifiers,axis=1,inplace=True)

# Merge the qualifiers on to the event dataframe

In [8]:
# reorder columns so that the most used columns are towards the left of the dataframe
column_order = qualifiers.notnull().sum().sort_values(ascending=False).index
qualifiers = qualifiers[column_order].copy()
# drop index to prepare for a merge
qualifiers.reset_index(drop=False,inplace=True)
# merge events and qualifiers
df_events = df_events.merge(qualifiers,left_on='event_id',right_on='event_id',how='left',validate='1:1')

# Add a flag for the direction of play (event dataframe)

In [9]:
# add flag for whether the direction of play is right to left for the team
direction_of_play = df_events.loc[df_events.direction_of_play.notnull(), ['game_id','team_id','period_id','direction_of_play']]
df_events.drop('direction_of_play',axis=1,inplace=True)
direction_of_play['right_to_left'] = direction_of_play.direction_of_play=='Right to Left' 
direction_of_play.drop('direction_of_play',axis=1,inplace=True)
df_events = df_events.merge(direction_of_play, on=['game_id', 'team_id', 'period_id'], how='left', validate='m:1')

# Format the event dataframe

In [10]:
# convert object column to datetime
df_events['timestamp'] = df_events['timestamp'].apply(pd.to_datetime)
# sort the dataframe
df_events.sort_values(['game_id','period_id' ,'min', 'sec', 'timestamp'],inplace=True)
# remove deleted events
df_events = df_events[df_events.type != 'deleted_event'].copy()
# remove empty columns
df_events.dropna(how='all', axis=1, inplace=True)

# Show the results

In [11]:
df_events.head()

,game_id,team_event_id,event_id,team_id,player_id,period_id,min,sec,timestamp,type_id,...,hit_woodwork,shot_high_missed,shot_location_box_deepright,injured_player_id,goalkeeper_parried_safe,not_past_goal_line,free_kick,shot_location_small_boxcentre,injury,right_to_left
2,360481,2,2036897618,30,None,1,0,0,2011-08-21 16:00:38.967,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,360481,2,336246484,43,None,1,0,0,2011-08-21 16:00:39.132,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,360481,3,1372839298,43,37572,1,0,1,2011-08-21 16:00:40.179,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
10,360481,7,1962550717,43,42593,1,0,19,2011-08-21 16:00:58.445,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
5,360481,4,978322590,43,20664,1,0,2,2011-08-21 16:00:41.585,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [12]:
df_events.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1650 entries, 2 to 1668
Data columns (total 119 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   game_id                         1650 non-null   object        
 1   team_event_id                   1650 non-null   object        
 2   event_id                        1650 non-null   object        
 3   team_id                         1650 non-null   object        
 4   player_id                       1628 non-null   object        
 5   period_id                       1650 non-null   object        
 6   min                             1650 non-null   object        
 7   sec                             1650 non-null   object        
 8   timestamp                       1650 non-null   datetime64[ns]
 9   type_id                         1650 non-null   object        
 10  outcome                         1650 non-null   object        
 11  x  